In [1]:
import os
import sys

import numpy as np
import pandas as pd
from sqlalchemy import create_engine


sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('../..'))))
from strategy import EquityStrategy
from tester import Tester


base_path = os.path.abspath('../../..')
data_path = os.path.join(base_path, 'data')
database_path = os.path.join(data_path, 'database')
strategy_path = os.path.join(base_path, 'strategy')
check_path = os.path.join(strategy_path, 'check')
sys.path.append(strategy_path)

In [2]:
from strategy import CommodityStrategy, EquityStrategy, IRStrategy, EmergingStrategy
from tester import Tester

In [3]:
eindex_path = os.path.join(data_path, 'totindex.csv')
eindex1_path = os.path.join(data_path, 'priceindex.csv')
efuture_path = os.path.join(data_path, 'FutGenratio1.csv')

In [4]:
total_ret_idx = pd.read_csv(eindex_path, header=0, index_col=0, parse_dates=True)
total_ret = total_ret_idx.pct_change(1)
prc_idx = pd.read_csv(eindex1_path, header=0, index_col=0, parse_dates=True)
prc_ret = prc_idx.pct_change(1)
fut_idx = pd.read_csv(efuture_path, header=0, index_col=0, parse_dates=True)
fut_idx.index = pd.to_datetime(fut_idx.index, unit='D', origin=pd.Timestamp('1899-12-30'))
fut_ret = fut_idx.pct_change(1)

# 만약에 eindex에 빈 정보가 있으면 eindex1으로 대체
no_data_dates = total_ret[(total_ret.isnull().sum(axis=1) > 0).values].index
total_ret.loc[no_data_dates] = prc_ret.loc[no_data_dates]

# 2007년까지는 eindex 2008년부터는 future 사용
ERet = pd.concat([total_ret.loc[:'2007-12-31'], fut_ret.loc['2008-01-01':]], axis=0)
ERet.dropna(inplace=True)
Eindex = (1. + ERet).cumprod()

ERet.index.name = 'tdate'
Eindex.index.name = 'tdate'

ERet.columns.name = 'ticker'
Eindex.columns.name = 'ticker'

In [5]:
growthvalue = pd.read_csv(os.path.join(data_path, "growthvalue.csv"), index_col=0, parse_dates=True)
growthvalue.index.name = 'tdate'
growthvalue.columns.name = 'ticker'

In [6]:
minobs1=12
nopos=0.4
cs=0.35
per=3

In [7]:
i = 0

In [8]:
ret = ERet
index = Eindex

In [9]:
class EST(EquityStrategy):
    def __init__(self, strategy_name, asset_type):
        super().__init__(strategy_name=strategy_name, asset_type=asset_type)
        self.growthvalue = None

    def load_strategy_data(self, table='datastream', origin='EPS'):
        self.growthvalue = self._load_strategy_data(table=table, origin=origin)

    def calculate_signal(self, minobs1=12, nopos=0.4, cs=0.35, per=3):
        for i in range(2):
            if i == 0:
                RET = self.ret.loc[:, ['SPX', 'TSX', 'FTSE', 'DAX', 'CAC', 'SMI', 'MIB', 'IBEX', 'OMX', 'AEX']]
            else:
                RET = self.ret.loc[:, ['NKY', 'AS51', 'HSI', "SG"]]

            self.logger.info('[STEP 3] CACULATE SIGNAL')
            GV = self.growthvalue
            sent = pd.DataFrame(GV.iloc[per:].to_numpy() / GV.iloc[:-per].to_numpy(),
                                columns=GV.columns,
                                index=GV.iloc[per:].index)

            b_columns = []
            v_columns = []

            for column in sent.columns:
                if column[-1].upper() == "V":
                    v_columns.append(column)
                else:
                    b_columns.append(column)

            sent = sent - 1
            sent1 = pd.DataFrame(
                sent.loc[:, b_columns].to_numpy() - sent.loc[:, v_columns].to_numpy(),
                columns=b_columns,
                index=sent.index)
            sent1 = sent1[RET.columns]
            RV = sent1.sub(sent1.mean(axis=1), axis=0)

            # 4. Rank
            pctrank = lambda x: pd.Series(x).rank(pct=True).iloc[-1]
            RVrank = RV.expanding().apply(pctrank, raw=True)  # it takes some time
            RVrank = RVrank.iloc[minobs1 - 1:, ]

            # 5. Long Short
            RV1 = RV.iloc[minobs1 - 1:, ]
            truecount = (RV1.notnull().sum(axis=1) * cs).apply(round)
            tiebreaker = RV1.rolling(5).mean().fillna(0) * 0.0000001

            # 1. Cross sectional
            CSRV = (RV1 + tiebreaker).rank(axis=1, method='first')  # Short
            CSRV1 = (-1 * RV1 - 1 * tiebreaker).rank(axis=1, method='first')  # Long

            CSRVpos = CSRV.fillna(0) * 0
            CSRVpos[CSRV.apply(lambda x: x <= truecount, axis=0)] = -1
            CSRVpos[CSRV1.apply(lambda x: x <= truecount, axis=0)] = 1

            # Final CS signal
            CSRV = CSRVpos
            CSRV.fillna(0, inplace=True)

            # 2. Time Series
            TSRV = RVrank.fillna(0) * 0
            TSRV[RVrank > nopos + (1 - nopos) / 2] = 1  # Long
            TSRV[RVrank < (1 - nopos) / 2] = -1  # Short

            if i == 0:
                TSRVrun1 = TSRV
                CSRVrun1 = CSRV
            else:
                TSRVrun2 = TSRV
                CSRVrun2 = CSRV

        TSRV = pd.concat([TSRVrun1, TSRVrun2], axis=1)
        CSRV = pd.concat([CSRVrun1, CSRVrun2], axis=1)

        TSRV = TSRV[self.ret.columns]
        CSRV = CSRV[self.ret.columns]

        # Align dates with Return DataFrame
        self.TSRV = TSRV.loc[self.ret.index].fillna(method='ffill').dropna(how='all')
        self.CSRV = CSRV.loc[self.ret.index].fillna(method='ffill').dropna(how='all')

        # Align dates with each other
        if self.TSRV.index[0] > self.CSRV.index[0]:
            self.CSRV = self.CSRV.loc[self.TSRV.index[0]:]
        else:
            self.TSRV = self.TSRV.loc[self.CSRV.index[0]:]

In [10]:
est = EST(strategy_name="EST", asset_type="EQUITY")
est.index = Eindex.copy()
est.ret = ERet.copy()
# est.load_index_and_return(from_db=False, save_file=False)

growthvalue = pd.read_csv(os.path.join(data_path, "growthvalue.csv"), index_col=0, parse_dates=True)
growthvalue.index.name = 'tdate'
growthvalue.columns.name = 'ticker'
est.growthvalue = growthvalue

# est.load_strategy_data(table='FS', origin='growthvalue')
est.set_rebalance_period(ts_freq='month', cs_freq='month')  # rebalance_day: monday = 0, sunday = 6
est.calculate_signal(minobs1=12, nopos=0.4, cs=0.35, per=3)
est.set_portfolio_parameter(cs_strategy_type="notional")
# est.make_portfolio()

2019-12-26 17:03:37,179 - EST - INFO - [STEP 0] START LOGGING EST
2019-12-26 17:03:37,186 - EST - INFO - [STEP 2] SET REBALANCE PERIOD
2019-12-26 17:03:37,188 - EST - INFO - [STEP 3] CACULATE SIGNAL
2019-12-26 17:03:37,684 - EST - INFO - [STEP 3] CACULATE SIGNAL
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:80: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:81: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
2019-12-26 17:03:37,905 - EST - INFO - [S

In [11]:
est_TSRV = pd.read_csv('../past/est_tsrv.csv', index_col=[0])
est_CSRV = pd.read_csv('../past/est_csrv.csv', index_col=[0])

est_TSRV.index = pd.to_datetime(est_TSRV.index)
est_CSRV.index = pd.to_datetime(est_CSRV.index)

In [12]:
TSRV_DIFF = (est_TSRV - est.TSRV)

In [13]:
TSRV_DIFF[np.abs(TSRV_DIFF) > 0.001].sum(axis=0)

SPX     0.0
TSX     0.0
FTSE    0.0
DAX     0.0
CAC     0.0
SMI     0.0
MIB     0.0
IBEX    0.0
OMX     0.0
AEX     0.0
NKY     0.0
AS51    0.0
HSI     0.0
SG      0.0
dtype: float64

In [14]:
CSRV_DIFF = (est_CSRV - est.CSRV)

In [15]:
CSRV_DIFF[np.abs(CSRV_DIFF) > 0.001].sum(axis=0)

SPX     0.0
TSX     0.0
FTSE    0.0
DAX     0.0
CAC     0.0
SMI     0.0
MIB     0.0
IBEX    0.0
OMX     0.0
AEX     0.0
NKY     0.0
AS51    0.0
HSI     0.0
SG      0.0
dtype: float64

In [16]:
est.make_portfolio()

2019-12-26 17:03:37,971 - EST - INFO - [STEP 5] MAKE PORTFOLIO
2019-12-26 17:03:37,972 - EST - INFO - [STEP 5 - 1] CALCULATE VOLATILITY
2019-12-26 17:04:38,083 - EST - INFO - [STEP 5 - 2] MAKE TS POSITION
2019-12-26 17:04:38,083 - EST - INFO - [STEP 5 - 2 - 1] ALIGN TS POSITION WITH REBALANCE DAY
2019-12-26 17:04:40,649 - EST - INFO - [STEP 5 - 2 - 2] TARGET VOL CONTROL to TS POSITION
2019-12-26 17:04:40,656 - EST - INFO - [STEP 5 - 2 - 3] STRATEGY LEVEL VOL CONTROL to TS POSITION
2019-12-26 17:04:41,741 - EST - INFO - [STEP 5 - 2 - 4] MAKE FINAL TS POSITION
2019-12-26 17:04:41,744 - EST - INFO - [STEP 5 - 3] MAKE CS POSITION
2019-12-26 17:04:41,745 - EST - INFO - [STEP 5 - 3 - 1] NO TARGET VOL CONTROL to CS POSITION
2019-12-26 17:04:41,745 - EST - INFO - [STEP 5 - 3 - 2] ALIGN CS POSITION WITH REBALANCE DAY
2019-12-26 17:04:46,015 - EST - INFO - [STEP 5 - 3 - 3] STRATEGY LEVEL VOL CONTROL to CS POSITION
2019-12-26 17:04:47,261 - EST - INFO - [STEP 5 - 3 - 4] MAKE FINAL CS POSITION
201

### GRP HELPER 확인

In [17]:
est.TSRV

,SPX,TSX,FTSE,DAX,CAC,SMI,MIB,IBEX,OMX,AEX,NKY,AS51,HSI,SG
tdate,,,,,,,,,,,,,,
1989-03-31,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,1.0,-1.0,0.0
1989-04-03,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,1.0,-1.0,0.0
1989-04-04,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,1.0,-1.0,0.0
1989-04-05,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,1.0,-1.0,0.0
1989-04-06,0.0,1.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,1.0,-1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-05-03,1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,0.0,-1.0,1.0
2018-05-04,1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,0.0,-1.0,1.0
2018-05-07,1.0,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,1.0,1.0,0.0,0.0,-1.0,1.0
